In [1]:
import os
import pandas as pd

# Define the paths to your dataset
dataset_paths = [
    r'C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\test',
    r'C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\train',
    r'C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val'
]

# Initialize empty lists to store full paths of filenames and corresponding classes
full_paths = []
classes = []

# Iterate over each dataset path
for dataset_path in dataset_paths:
    # List all subdirectories in the dataset_path
    subdirectories = [os.path.join(dataset_path, d) for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]
    
    # Loop through each subdirectory
    for subdir in subdirectories:
        # Get the class name from the last part of the path
        class_name = os.path.basename(subdir)
        
        # List all files in the subdirectory
        files = [os.path.join(subdir, f) for f in os.listdir(subdir) if os.path.isfile(os.path.join(subdir, f))]
        
        # Append full paths of filenames and their corresponding classes to the lists
        full_paths.extend(files)
        classes.extend([class_name] * len(files))

# Create a dataframe from the lists
df = pd.DataFrame({'filename': full_paths, 'class': classes})

# Get class counts
class_counts = df['class'].value_counts()

# Display the class counts
print(class_counts)


class
Melanocytic nevus       505
Melanoma                330
Basal cell carcinoma    248
Benign keratosis        168
Actinic keratosis       167
Name: count, dtype: int64


In [2]:
df

,filename,class
0,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
1,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
2,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
3,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
4,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\tes...,Actinic keratosis
...,...,...
1413,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val...,Melanoma
1414,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val...,Melanoma
1415,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val...,Melanoma
1416,C:\Users\T 480\NEW_ISIC - 2019\ISIC - 2019\val...,Melanoma


In [5]:
pip install torchvision


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   - -------------------------------------- 0.0/1.2 MB 487.6 kB/s eta 0:00:03
   ---- ----------------------------------- 0.1/1.2 MB 1.2 MB/s eta 0:00:01
   ----------- ---------------------------- 0.3/1.2 MB 2.2 MB/s eta 0:00:01
   ------------------- -------------------- 0.6/1.2 MB 2.7 MB/s eta 0:00:01
   --------------------------- ------------ 0.8/1.2 MB 3.2 MB/s eta 0:00:01
   --------------------------------- ------ 1.0/1.2 MB 3.2 MB/s eta 0:00:01
   ------------------------------------- -- 1.1/1.2 MB 3.3 MB/s eta 0:00:01
   ---------------------------------------  1.2/1.2 MB 3.2 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 3.0 MB/s eta 0:00:00


In [3]:
pip install torch

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as dset
import torchvision.transforms as transforms
import torchvision.utils as vutils
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt
import os

# Set random seed for reproducibility
manual_seed = 999
torch.manual_seed(manual_seed)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class BalancedDataset(Dataset):
    def __init__(self, root, transform=None):
        self.root = root
        self.transform = transform
        self.classes = [d for d in os.listdir(root) if os.path.isdir(os.path.join(root, d))]

        self.class_counts = {}
        for cls in self.classes:
            cls_dir = os.path.join(root, cls)
            cls_images = [img for img in os.listdir(cls_dir)]
            if len(cls_images) == 0:
                print(f"No images found in directory: {cls_dir}")
            else:
                self.class_counts[cls] = len(cls_images)

        if len(self.class_counts) == 0:
            raise ValueError("No images found in any class directory")

        max_count = max(self.class_counts.values())

        self.samples = []
        for cls, count in self.class_counts.items():
            cls_dir = os.path.join(root, cls)
            cls_images = [img for img in os.listdir(cls_dir)]
            self.samples.extend([(cls, img) for img in np.random.choice(cls_images, max_count, replace=True)])

    def __getitem__(self, index):
        cls, img_name = self.samples[index]
        img_path = os.path.join(self.root, cls, img_name)
        image = Image.open(img_path).convert("RGB")
        if self.transform is not None:
            image = self.transform(image)
        return image

    def __len__(self):
        return len(self.samples)


# Define Generator and Discriminator architectures
# Code for Generator and Discriminator can be added here

# Define the training loop
# Code for training loop can be added here

# Define the transformations for the images
transform = transforms.Compose([
    transforms.Resize(64),
    transforms.CenterCrop(64),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

# Set the root directory of your dataset
root_dir = r"C:\Users\T 480\Isic\reduced_images"

# Create a balanced dataset instance
balanced_dataset = BalancedDataset(root=root_dir, transform=transform)

# Create a DataLoader for the balanced dataset
dataloader = DataLoader(balanced_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

# Train your DCGAN using the DataLoader

# Optionally, you can visualize generated images during training

# Once trained, you can use the Generator to generate synthetic samples for the underrepresented classes


ValueError: No images found in any class directory

In [9]:
import os
import pandas as pd

# Directory path
directory = r'C:\Users\T 480\NEW_ISIC - 2019\new2019'

# Initialize lists to store data
file_paths = []
classes = []

# Traverse through the directory
for root, dirs, files in os.walk(directory):
    # Extract class name from the current directory
    class_name = os.path.basename(root)
    
    # Iterate through files
    for file in files:
        # Append file path
        file_paths.append(os.path.join(root, file))
        # Append class
        classes.append(class_name)

# Create DataFrame
df = pd.DataFrame({'File_Path': file_paths, 'Class': classes})

# Display the DataFrame
print(df.head())


                                           File_Path              Class
0  C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...  Actinic keratosis
1  C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...  Actinic keratosis
2  C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...  Actinic keratosis
3  C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...  Actinic keratosis
4  C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...  Actinic keratosis


In [10]:
import os
import pandas as pd

# Directory path
directory = r'C:\Users\T 480\NEW_ISIC - 2019\new2019'

# Initialize lists to store data
file_paths = []
classes = []

# Traverse through the directory
for root, dirs, files in os.walk(directory):
    # Extract class name from the current directory
    class_name = os.path.basename(root)
    
    # Iterate through files
    for file in files:
        # Append file path
        file_paths.append(os.path.join(root, file))
        # Append class
        classes.append(class_name)

# Create DataFrame
df = pd.DataFrame({'File_Path': file_paths, 'Class': classes})

# Get class counts
class_counts = df['Class'].value_counts()

# Display class counts
print(class_counts)


Class
Melanocytic nevus       505
Melanoma                330
Basal cell carcinoma    248
Benign keratosis        168
Actinic keratosis       167
Name: count, dtype: int64


In [11]:
df

,File_Path,Class
0,C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...,Actinic keratosis
1,C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...,Actinic keratosis
2,C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...,Actinic keratosis
3,C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...,Actinic keratosis
4,C:\Users\T 480\NEW_ISIC - 2019\new2019\Actinic...,Actinic keratosis
...,...,...
1413,C:\Users\T 480\NEW_ISIC - 2019\new2019\Melanom...,Melanoma
1414,C:\Users\T 480\NEW_ISIC - 2019\new2019\Melanom...,Melanoma
1415,C:\Users\T 480\NEW_ISIC - 2019\new2019\Melanom...,Melanoma
1416,C:\Users\T 480\NEW_ISIC - 2019\new2019\Melanom...,Melanoma


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the architecture for the generator
def build_generator(latent_dim):
    model = models.Sequential()
    model.add(layers.Dense(128, input_dim=latent_dim))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(784, activation='tanh'))
    model.add(layers.Reshape((28, 28, 1)))
    return model

# Define the architecture for the discriminator
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Flatten(input_shape=(28, 28, 1)))
    model.add(layers.Dense(512))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(256))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dense(1, activation='sigmoid'))
    return model

# Define the GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False
    model = models.Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Define the training loop
def train_gan(generator, discriminator, gan, data_generator, latent_dim, epochs=100, batch_size=128):
    for epoch in range(epochs):
        for _ in range(len(data_generator)):
            # Get batch of real images
            real_images, _ = next(data_generator)

            # Generate batch of fake images
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            fake_images = generator.predict(noise)

            # Train discriminator
            discriminator.trainable = True
            d_loss_real = discriminator.train_on_batch(real_images, np.ones((batch_size, 1)))
            d_loss_fake = discriminator.train_on_batch(fake_images, np.zeros((batch_size, 1)))
            d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

            # Train generator
            noise = np.random.normal(0, 1, (batch_size, latent_dim))
            discriminator.trainable = False
            g_loss = gan.train_on_batch(noise, np.ones((batch_size, 1)))

            # Print the progress
            print(f"Epoch {epoch + 1}, D Loss: {d_loss}, G Loss: {g_loss}")

# Define your data directory
data_dir = r"C:\Users\T 480\NEW_ISIC - 2019\new2019"

# Set hyperparameters
latent_dim = 100
epochs = 100
batch_size = 128

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load and augment your data
data_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(28, 28),
    batch_size=batch_size,
    class_mode='binary'
)

# Build and compile the discriminator
discriminator = build_discriminator()
discriminator.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Build the generator
generator = build_generator(latent_dim)

# Build the GAN model
gan = build_gan(generator, discriminator)
gan.compile(optimizer='adam', loss='binary_crossentropy')

# Train the GAN
train_gan(generator, discriminator, gan, data_generator, latent_dim, epochs, batch_size)


Found 1418 images belonging to 5 classes.
